<H2> Importing Libaries and Data</H2>


In [1]:
import os
import torch
from torch import nn, optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.ToTensor()  # Convert image to tensor
])

parent_dir = os.path.dirname(os.getcwd())

# Construct the path to 'Data/Colorectal Cancer'
data_dir = os.path.join(parent_dir, 'Data', 'Colorectal Cancer')
dataset = ImageFolder(root=data_dir, transform=transform)

train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


<H2> Inspecting Data Sample and Loaders</H2>

In [2]:
data_iter = iter(train_loader)
images, labels = next(data_iter)

print(f"Total images: {len(dataset)}")
print(f"Training set: {len(train_dataset)} images")
print(f"Testing set: {len(test_dataset)} images")
print(f"Shape of the tensor: {images[0].shape}")
print(f"Classes :{dataset.classes}")

Total images: 6000
Training set: 4200 images
Testing set: 1800 images
Shape of the tensor: torch.Size([3, 224, 224])
Classes :['MUS', 'NORM', 'STR']


<H2> Compute Mean and STD of Training set for Normalization </H2>

In [3]:
mean = 0.0
std = 0.0
nb_samples = 0

# Iterate over the training set
for images, _ in tqdm(train_loader):
    batch_samples = images.size(0)  # Number of images in batch
    images = images.view(batch_samples, images.size(1), -1)  # Flatten HxW dimensions
    mean += images.mean(2).sum(0)  # Sum means across all images
    std += images.std(2).sum(0)    # Sum stds across all images
    nb_samples += batch_samples    # Accumulate total number of samples

# Compute mean and std across the entire training set
mean /= nb_samples
std /= nb_samples

print(f"Mean: {mean}")
print(f"Standard Deviation: {std}")

100%|██████████| 132/132 [00:05<00:00, 24.28it/s]

Mean: tensor([0.7624, 0.5233, 0.7113])
Standard Deviation: tensor([0.1379, 0.1719, 0.1269])


<H2> Normalize Dataset </H2>

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

<H2> Create ResNet-50 Class </H2>

In [5]:
class CustomResNet50(nn.Module):
    def __init__(self, num_classes=3):
        super(CustomResNet50, self).__init__()
        
        # Load a ResNet-50 model with no pre-trained weights
        self.model = models.resnet50(weights=None)
        
        # Modify the final fully connected layer to output 'num_classes' classes
        in_features = self.model.fc.in_features
        self.model.fc = nn.Linear(in_features, num_classes)
        
    def forward(self, x):
        return self.model(x)

# Instantiate the model
num_classes = len(dataset.classes)
model = CustomResNet50(num_classes=num_classes)


In [52]:
# Hyperparameters to experiment with
batch_sizes = [16, 32, 64]
learning_rates = [0.01, 0.001, 0.0001]
num_epochs = 10

# Loss function
criterion = nn.CrossEntropyLoss()

# Iterate through different batch sizes
for batch_size in batch_sizes:
    # DataLoader for training and testing sets with the current batch size
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Iterate through different learning rates
    for lr in learning_rates:
        # Initialize model and move to the device
        model = CustomResNet50(num_classes=3).to(device)

        # Optimizer for the current learning rate
        optimizer = optim.Adam(model.parameters(), lr=lr)

        # Track metrics for each combination
        train_acc, test_acc, train_loss, test_loss = [], [], [], []

        print(f"\nTraining with Batch Size: {batch_size}, Learning Rate: {lr}")

        # Training loop
        for epoch in range(num_epochs):
            # Training phase
            model.train()
            running_loss, running_corrects = 0.0, 0

            # Adding tqdm loading bar
            for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()  # Zero gradients
                outputs = model(inputs)  # Forward pass
                loss = criterion(outputs, labels)  # Compute loss
                loss.backward()  # Backpropagation
                optimizer.step()  # Update weights

                # Update training metrics
                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            # Calculate epoch metrics for training
            epoch_train_loss = running_loss / len(train_dataset)
            epoch_train_acc = running_corrects.double() / len(train_dataset)
            train_loss.append(epoch_train_loss)
            train_acc.append(epoch_train_acc.item())

            # Testing phase
            model.eval()
            running_loss, running_corrects = 0.0, 0

            with torch.no_grad():
                for inputs, labels in test_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    # Update testing metrics
                    _, preds = torch.max(outputs, 1)
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

            # Calculate epoch metrics for testing
            epoch_test_loss = running_loss / len(test_dataset)
            epoch_test_acc = running_corrects.double() / len(test_dataset)
            test_loss.append(epoch_test_loss)
            test_acc.append(epoch_test_acc.item())

            # Print epoch metrics
            print(f"Epoch {epoch+1}/{num_epochs}, "
                  f"Train Loss: {train_loss[-1]:.4f}, Train Acc: {train_acc[-1]:.4f}, "
                  f"Test Loss: {test_loss[-1]:.4f}, Test Acc: {test_acc[-1]:.4f}")

        # Final summary for each combination
        print(f"\nFinal Results for Batch Size: {batch_size}, Learning Rate: {lr}")
        print(f"Train Accuracy: {train_acc[-1]:.4f}, Test Accuracy: {test_acc[-1]:.4f}\n")


Training with Batch Size: 16, Learning Rate: 0.01


Epoch 1/10, Train Loss: 1.2610, Train Acc: 0.5198, Test Loss: 0.6821, Test Acc: 0.6561


Epoch 2/10, Train Loss: 0.6537, Train Acc: 0.6881, Test Loss: 1.4021, Test Acc: 0.5422


Epoch 3/10, Train Loss: 0.5684, Train Acc: 0.7669, Test Loss: 0.6896, Test Acc: 0.6989


Epoch 4/10, Train Loss: 0.4740, Train Acc: 0.8190, Test Loss: 0.7868, Test Acc: 0.6694


...

Epoch 6/10, Train Loss: 0.2067, Train Acc: 0.9221, Test Loss: 0.5846, Test Acc: 0.7917


Epoch 7/10, Train Loss: 0.1773, Train Acc: 0.9331, Test Loss: 0.3801, Test Acc: 0.8622


Epoch 8/10, Train Loss: 0.1835, Train Acc: 0.9321, Test Loss: 1.6285, Test Acc: 0.6556


Epoch 9/10, Train Loss: 0.1394, Train Acc: 0.9476, Test Loss: 0.8154, Test Acc: 0.8072


Epoch 10/10, Train Loss: 0.1196, Train Acc: 0.9529, Test Loss: 0.7811, Test Acc: 0.8033

Final Results for Batch Size: 64, Learning Rate: 0.0001
Train Accuracy: 0.9529, Test Accuracy: 0.8033



<H2> Testing Batch Size 32, Lr = 0.001 </H2>

In [53]:
import torch
from torch import nn, optim
from tqdm import tqdm

# Set hyperparameters
batch_size = 32
learning_rate = 0.001
num_epochs = 15

# Check if a GPU is available and set device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# DataLoader for training and testing sets with batch size 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize model and move to the device
model = CustomResNet50(num_classes=3).to(device)

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer with weight decay for regularization
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

# Learning rate scheduler (Cosine Annealing)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

# Track metrics
train_acc, test_acc, train_loss, test_loss = [], [], [], []

# Training loop
for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss, running_corrects = 0.0, 0

    # Adding tqdm loading bar
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # Zero gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

        # Update training metrics
        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    # Calculate epoch metrics for training
    epoch_train_loss = running_loss / len(train_dataset)
    epoch_train_acc = running_corrects.double() / len(train_dataset)
    train_loss.append(epoch_train_loss)
    train_acc.append(epoch_train_acc.item())

    # Testing phase
    model.eval()
    running_loss, running_corrects = 0.0, 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Update testing metrics
            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

    # Calculate epoch metrics for testing
    epoch_test_loss = running_loss / len(test_dataset)
    epoch_test_acc = running_corrects.double() / len(test_dataset)
    test_loss.append(epoch_test_loss)
    test_acc.append(epoch_test_acc.item())

    # Step the scheduler
    scheduler.step()

    # Print epoch metrics
    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Train Loss: {train_loss[-1]:.4f}, Train Acc: {train_acc[-1]:.4f}, "
          f"Test Loss: {test_loss[-1]:.4f}, Test Acc: {test_acc[-1]:.4f}")

# Final Results
print(f"\nFinal Results for Batch Size: {batch_size}, Learning Rate: {learning_rate}")
print(f"Train Accuracy: {train_acc[-1]:.4f}, Test Accuracy:")


Epoch 1/15, Train Loss: 0.6803, Train Acc: 0.7486, Test Loss: 1.2287, Test Acc: 0.6189


Epoch 2/15, Train Loss: 0.3834, Train Acc: 0.8626, Test Loss: 0.4584, Test Acc: 0.8089


Epoch 3/15, Train Loss: 0.3037, Train Acc: 0.8867, Test Loss: 3.1119, Test Acc: 0.7211


Epoch 4/15, Train Loss: 0.2361, Train Acc: 0.9138, Test Loss: 1.7364, Test Acc: 0.6217


Epoch 5/15, Train Loss: 0.2369, Train Acc: 0.9112, Test Loss: 0.5975, Test Acc: 0.8211


...

Epoch 11/15, Train Loss: 0.0525, Train Acc: 0.9817, Test Loss: 0.0712, Test Acc: 0.9783


Epoch 12/15, Train Loss: 0.0388, Train Acc: 0.9876, Test Loss: 0.0570, Test Acc: 0.9800


Epoch 13/15, Train Loss: 0.0347, Train Acc: 0.9883, Test Loss: 0.0567, Test Acc: 0.9756


Epoch 14/15, Train Loss: 0.0219, Train Acc: 0.9948, Test Loss: 0.0372, Test Acc: 0.9889


Epoch 15/15, Train Loss: 0.0204, Train Acc: 0.9943, Test Loss: 0.0433, Test Acc: 0.9878

Final Results for Batch Size: 32, Learning Rate: 0.001
Train Accuracy: 0.9943, Test Accuracy:


In [ ]:
from sklearn.model_selection import KFold
# Hyperparameters
batch_size = 32
learning_rate = 0.001
num_epochs = 10
num_folds = 5  # Number of folds for k-fold cross-validation

# Check if a GPU is available and set device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize model
def create_model():
    model = CustomResNet50(num_classes=3).to(device)
    return model

# Loss function
criterion = nn.CrossEntropyLoss()

# K-Fold Cross-Validation
kfold = KFold(n_splits=num_folds, shuffle=True)

# Track metrics for each fold
fold_train_acc, fold_test_acc, fold_train_loss, fold_test_loss = [], [], [], []

print(f"Starting {num_folds}-Fold Cross-Validation...\n")

# K-Fold Cross-Validation Loop
for fold, (train_ids, val_ids) in enumerate(kfold.split(dataset)):
    print(f"\nFold {fold + 1}/{num_folds}")
    
    # Sample subsets for training and validation
    train_subsampler = Subset(dataset, train_ids)
    val_subsampler = Subset(dataset, val_ids)

    # DataLoader for the current fold
    train_loader = DataLoader(train_subsampler, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_subsampler, batch_size=batch_size, shuffle=False)

    # Create a new model instance for each fold
    model = create_model()

    # Optimizer with weight decay
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

    # Learning rate scheduler (Cosine Annealing)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

    # Track metrics for the current fold
    train_acc, test_acc, train_loss, test_loss = [], [], [], []

    # Training loop for each fold
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss, running_corrects = 0.0, 0

        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()  # Zero gradients
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights

            # Update training metrics
            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        # Calculate epoch metrics for training
        epoch_train_loss = running_loss / len(train_subsampler)
        epoch_train_acc = running_corrects.double() / len(train_subsampler)
        train_loss.append(epoch_train_loss)
        train_acc.append(epoch_train_acc.item())

        # Validation phase
        model.eval()
        running_loss, running_corrects = 0.0, 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                # Update validation metrics
                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

        # Calculate epoch metrics for validation
        epoch_test_loss = running_loss / len(val_subsampler)
        epoch_test_acc = running_corrects.double() / len(val_subsampler)
        test_loss.append(epoch_test_loss)
        test_acc.append(epoch_test_acc.item())

        # Step the scheduler
        scheduler.step()

        # Print epoch metrics
        print(f"Epoch {epoch+1}/{num_epochs}, "
              f"Train Loss: {train_loss[-1]:.4f}, Train Acc: {train_acc[-1]:.4f}, "
              f"Val Loss: {test_loss[-1]:.4f}, Val Acc: {test_acc[-1]:.4f}")

    # Store fold results
    fold_train_acc.append(train_acc[-1])
    fold_test_acc.append(test_acc[-1])
    fold_train_loss.append(train_loss[-1])
    fold_test_loss.append(test_loss[-1])

    print(f"\nFold {fold + 1} Results - "
          f"Train Acc: {train_acc[-1]:.4f}, Val Acc: {test_acc[-1]:.4f}")

# Print average results across all folds
print(f"\n{k}-Fold Cross-Validation Complete!")
print(f"Average Train Acc: {np.mean(fold_train_acc):.4f}, Average Val Acc: {np.mean(fold_test_acc):.4f}")
print(f"Average Train Loss: {np.mean(fold_train_loss):.4f}, Average Val Loss: {np.mean(fold_test_loss):.4f}")


Starting 5-Fold Cross-Validation...


Fold 1/5


Epoch 1/10, Train Loss: 0.6172, Train Acc: 0.7550, Val Loss: 3.7532, Val Acc: 0.4800


Epoch 2/10, Train Loss: 0.3698, Train Acc: 0.8594, Val Loss: 0.8514, Val Acc: 0.6967


Epoch 3/10, Train Loss: 0.2893, Train Acc: 0.8883, Val Loss: 0.4648, Val Acc: 0.8083


Epoch 4/10, Train Loss: 0.2379, Train Acc: 0.9067, Val Loss: 0.3100, Val Acc: 0.8908


...

Epoch 7/10, Train Loss: 0.0890, Train Acc: 0.9694, Val Loss: 5.3339, Val Acc: 0.3842


Epoch 8/10, Train Loss: 0.0690, Train Acc: 0.9750, Val Loss: 0.3282, Val Acc: 0.8925


Epoch 9/10, Train Loss: 0.0466, Train Acc: 0.9835, Val Loss: 0.2919, Val Acc: 0.9117


Epoch 10/10, Train Loss: 0.0310, Train Acc: 0.9904, Val Loss: 0.0333, Val Acc: 0.9892

Fold 5 Results - Train Acc: 0.9904, Val Acc: 0.9892


NameError: name 'k' is not defined

In [8]:
avg_train_acc = (0.9879 + 0.9885 + 0.9869 + 0.9871 + 0.9904) / 5
avg_val_acc = (0.9892 + 0.9883 + 0.9900 + 0.9933 + 0.9892) / 5

print(f"Average Train Accuracy: {avg_train_acc:.4f}")
print(f"Average Validation Accuracy: {avg_val_acc:.4f}")

Average Train Accuracy: 0.9882
Average Validation Accuracy: 0.9900


In [ ]:
import torch
from torch import nn, optim
from tqdm import tqdm

# Set hyperparameters
batch_size = 16
learning_rate = 0.001
num_epochs = 15

# Check if a GPU is available and set device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# DataLoader for training and testing sets with batch size 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize model and move to the device
model = CustomResNet50(num_classes=3).to(device)

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer with weight decay for regularization
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

# Learning rate scheduler (Cosine Annealing)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

# Track metrics
train_acc, test_acc, train_loss, test_loss = [], [], [], []

# Training loop
for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss, running_corrects = 0.0, 0

    # Adding tqdm loading bar
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # Zero gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

        # Update training metrics
        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    # Calculate epoch metrics for training
    epoch_train_loss = running_loss / len(train_dataset)
    epoch_train_acc = running_corrects.double() / len(train_dataset)
    train_loss.append(epoch_train_loss)
    train_acc.append(epoch_train_acc.item())

    # Testing phase
    model.eval()
    running_loss, running_corrects = 0.0, 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Update testing metrics
            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

    # Calculate epoch metrics for testing
    epoch_test_loss = running_loss / len(test_dataset)
    epoch_test_acc = running_corrects.double() / len(test_dataset)
    test_loss.append(epoch_test_loss)
    test_acc.append(epoch_test_acc.item())

    # Step the scheduler
    scheduler.step()

    # Print epoch metrics
    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Train Loss: {train_loss[-1]:.4f}, Train Acc: {train_acc[-1]:.4f}, "
          f"Test Loss: {test_loss[-1]:.4f}, Test Acc: {test_acc[-1]:.4f}")

# Final Results
print(f"\nFinal Results for Batch Size: {batch_size}, Learning Rate: {learning_rate}")
print(f"Train Accuracy: {train_acc[-1]:.4f}, Test Accuracy:")
